### Zaawansowane Metody Uczenia Maszynowego

#### Laboratorium 11

### Eksperyment

1. Weź dowolny zbiór danych dotyczący klasyfikacji.
2. Podziel wybrany zbiór na treningowy i testowy.
3. Dla pewnego procenta k obserwacji w zbiorze treningowym usuwamy etykiety. Rozważ różne wartośći k, np. 20%, 50%, 80%, 90%. Czyli 90% oznacza, że 90% obserwacji ma etykietę "?", zaś 10% obserwacji ma etykietę "0" lub "1".
4. Dopasuj modele: a) metoda naiwna: model bazujący tylko na obserwacjach gdzie została etykieta b) metoda semi-supervised: model bazujący na self-training, który wykorzystuje obserwacje bez etykiety.
5. Sprawdź oba modele na danych testowych.

Cały eksperyment powtórz dla przynajmniej dwóch zbiorów danych.

#### Zbiór danych: credit-g

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Pobieranie zbioru danych

In [ ]:
from sklearn.datasets import fetch_openml

df = fetch_openml(data_id=31)
y = df.target
X = df.data

y = y == "good"
y = y.astype(int)

Preprocessing danych

In [3]:
numerical_features = list(X.dtypes[(X.dtypes != 'object') & (X.dtypes != 'category')].index)
categorical_features = list(X.dtypes[(X.dtypes == 'object') | (X.dtypes == 'category')].index)

print('Numerical features:', numerical_features)
print('Categorical features:', categorical_features)

Numerical features: ['duration', 'credit_amount', 'installment_commitment', 'residence_since', 'age', 'existing_credits', 'num_dependents']
Categorical features: ['checking_status', 'credit_history', 'purpose', 'savings_status', 'employment', 'personal_status', 'other_parties', 'property_magnitude', 'other_payment_plans', 'housing', 'job', 'own_telephone', 'foreign_worker']


In [4]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

numerical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy = 'mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
])

preprocessor = ColumnTransformer([
    ('numerical', numerical_transformer, numerical_features),
    ('categorical', categorical_transformer, categorical_features)
])

Podział danych

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)


Eksperyment: metoda naiwna 

In [6]:
import random
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.semi_supervised import SelfTrainingClassifier


methods = [KNeighborsClassifier(), LogisticRegression(), DecisionTreeClassifier(), RandomForestClassifier()]
percent = [0.2, 0.5, 0.8, 0.9]
train_auc = []
test_auc = []
train_auc_semi = []
test_auc_semi = []
models = []
ks = []

for method in methods:
    for k in percent:
        X_train_k = X_train.sample(int(np.round((1-k)*y_train.size)))
        y_train_k = y_train[X_train_k.index]

        X_train_k_semi = X_train        
        y_train_k_semi = y_train

        pipeline = Pipeline([
            ('pre', preprocessor),
            ('tree', method)
        ])

        model = pipeline.fit(X_train_k, y_train_k)
        models.append(str(method))
        ks.append(k)
        train_auc.append((roc_auc_score(y_train_k, model.predict_proba(X_train_k)[:,1])))
        test_auc.append((roc_auc_score(y_test, model.predict_proba(X_test)[:,1])))

In [7]:
model_naiwny = pd.DataFrame({"k" : ks,
             "train_auc" : train_auc,
             "test_auc" : test_auc,
             "model" : models})

In [8]:
model_naiwny

,k,train_auc,test_auc,model
0,0.2,0.888893,0.727030,KNeighborsClassifier()
1,0.5,0.863310,0.687464,KNeighborsClassifier()
2,0.8,0.798627,0.653343,KNeighborsClassifier()
3,0.9,0.814116,0.630750,KNeighborsClassifier()
4,0.2,0.849429,0.755532,LogisticRegression()
5,0.5,0.855190,0.738153,LogisticRegression()
6,0.8,0.919146,0.728421,LogisticRegression()
7,0.9,0.984469,0.748812,LogisticRegression()
8,0.2,1.000000,0.578612,DecisionTreeClassifier()
9,0.5,1.000000,0.629881,DecisionTreeClassifier()
